---
jupyter: python3
title: "Neural Cellular Automaton"
format:
  html:
    cold-fold : true
execute:
  cache: true
---

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

EMOJI = "mage.png"

class Pattern:

    def __init__(self, path):
        ref = Image.open(path)
        ref.thumbnail((40, 40), Image.LANCZOS)
        ref = np.array(ref) / 255
        ref = np.pad(ref, ((30, 30), (30, 30), (0, 0)))
        ref = np.transpose(ref, (2, 1, 0))
        self.data = ref

    def shape(self):
        return self.data.shape

    def as_image(self):
        return np.transpose(self.data, (2, 1, 0))


img1 = Pattern(EMOJI)
plt.imshow(img1.as_image())

In [ ]:
import torch
from torch import nn
import torch.nn.functional
import time

device = "mps"

class NeuralCellularAutomata(nn.Module):

    def __init__(self):
        super().__init__()
        self.sequence = nn.Sequential(
            nn.Conv2d(16, 16 * 3, 3, groups=16, padding=1, bias=False), # Groups = 16 to treat all channels seperately # Why 3 ? Why not 3 * 3 ?
            nn.Conv2d(16 * 3, 128, 1),
            nn.ReLU(),
            nn.Conv2d(128, 16, 1),
        )
        #A commenter si pb
        self.sequence[-1].weight.data *= 0
        #nn.init.normal_(self.sequence[-1].weight, mean=0, std=0.01)

        id = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]])
        sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]) / 8 # Normalizing
        full_kernel = np.stack([id, sobel_x, sobel_x.T], axis=0)
        full_kernel = np.tile(full_kernel, [16, 1, 1])

        self.sequence[0].weight.data = torch.tensor(full_kernel, dtype=torch.float32).unsqueeze(1) # Addind dimension to match (N, C, H, W)
        self.sequence[0].weight.requires_grad = False

    def forward(self, steps):
        frames = [torch.zeros((1, 16, 100, 100)).to(device)]
        frames[0][0, 3:, 50, 50] = torch.ones(13)
        for step in range(steps):
            next_state = frames[-1] + self.sequence(frames[-1])# * bernouilli_mask
            #masked_state = self.alive_masking(next_state)
            frames.append(next_state)
        return frames

def bernouilli_mask(p = 0.5):
    mask = np.random.rand(1, 16, 100, 100) < p
    return torch.tensor(mask, device=device, dtype=torch.float32)

def normalize_grads(model):
  for p in model.parameters():
      p.grad = p.grad / (p.grad.norm() + 1e-8) if p.grad is not None else p.grad

def train(model : NeuralCellularAutomata, pt : Pattern, epochs = 100, lr=0.002):

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(params = model.parameters(), lr=lr)

    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        y_pred = model.forward(np.random.randint(64, 97))[-1]
        input = y_pred[0, :4, :, :].to(device)
        loss = loss_fn(input, torch.tensor(pt.data, device=device, dtype = torch.float32))
        optimizer.zero_grad()
        loss.backward()
        normalize_grads(model) # -----------> Absolument primordial - stagne sans (ou bcp plus long + incertain)
        optimizer.step()

        if (epoch+1) % (epochs / 10) == 0:
            print(f"Epoch : {epoch + 1}. Loss : {loss:.3f}. Speed : {(epochs/10) / (time.time() - start_time):.2f} epoch/s")
            start_time = time.time()

NCA0 = NeuralCellularAutomata().to(device)
PT0 = Pattern(EMOJI)

train(NCA0, PT0, 100)
torch.save(NCA0.state_dict(), "mage-model.pth")

In [ ]:
import math

frame_count = 200
frame_display = 20

with torch.inference_mode():
    t1 = time.time()
    frames = NCA0.forward(frame_count)
    print(f"Inference speed : {frame_count / (time.time() - t1):.2f} step/s")

frames = frames[::frame_display]

num_frames = len(frames)
cols = min(6, num_frames)
rows = int(np.ceil(num_frames / cols))

fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
if rows == 1 and cols == 1:
    axes = [axes]
elif rows == 1 or cols == 1:
    axes = axes.flatten()
else:
    axes = axes.flatten()

for i, frame in enumerate(frames):
    if i < len(axes):
        frame_data = np.transpose(frame[0, :4, :, :].cpu().numpy(), (2, 1, 0))
        frame_data = np.clip(frame_data, 0, 1)
        axes[i].imshow(frame_data)
        axes[i].axis('off')
        axes[i].set_title(f'Frame {i*frame_display}', fontsize=20)

for j in range(num_frames, len(axes)):
    axes[j].axis('off')

plt.tight_layout(pad=2.0)
plt.show()

In [ ]:
import imageio

with torch.inference_mode():
    frames = NCA0.forward(90)

split = 1
frames = frames[::split]

images = []
for frame in frames:
    img = np.transpose(frame[0, :4, :, :].cpu().numpy(), (2, 1, 0))
    img = np.clip(img, 0, 1)

    img = (img[:, :, :3] * 255).astype(np.uint8)
    images.append(Image.fromarray(img))

images[0].save(
    'nca_evolution.gif',
    save_all=True,
    append_images=images[1:],
    duration=100,
    loop=0
)